In [1]:
import pandas as pd
from catboost import Pool, CatBoostRegressor
print('Setup successful!')

Setup successful!


In [2]:
model = None
for ind in range(1, 13):
    name = f'train_{ind}.parquet'
    X_train = pd.read_parquet('/kaggle/input/avitotech-ml-cup-2024-splitted-train-dataset/' + name)
    y_train = X_train['target']
    X_train = X_train.drop(columns='target')
    data = Pool(X_train, y_train)
    model = CatBoostRegressor(
        iterations=69,
        learning_rate=0.1,
        depth=7
    ).fit(data, init_model=model, verbose=15)
print('model has been successfully fitted')
del X_train, y_train, data

0:	learn: 0.0664245	total: 954ms	remaining: 1m 4s
15:	learn: 0.0663967	total: 11.9s	remaining: 39.5s
30:	learn: 0.0663907	total: 23.3s	remaining: 28.6s
45:	learn: 0.0663869	total: 35.3s	remaining: 17.6s
60:	learn: 0.0663841	total: 46.4s	remaining: 6.09s
68:	learn: 0.0663829	total: 52.2s	remaining: 0us
0:	learn: 0.0680376	total: 989ms	remaining: 1m 7s
15:	learn: 0.0680324	total: 13.2s	remaining: 43.8s
30:	learn: 0.0680292	total: 24.8s	remaining: 30.4s
45:	learn: 0.0680265	total: 37s	remaining: 18.5s
60:	learn: 0.0680240	total: 48.1s	remaining: 6.31s
68:	learn: 0.0680228	total: 54s	remaining: 0us
0:	learn: 0.0679848	total: 1.15s	remaining: 1m 17s
15:	learn: 0.0679759	total: 12.8s	remaining: 42.3s
30:	learn: 0.0679733	total: 23.7s	remaining: 29.1s
45:	learn: 0.0679702	total: 35.8s	remaining: 17.9s
60:	learn: 0.0679683	total: 46.7s	remaining: 6.13s
68:	learn: 0.0679670	total: 52.7s	remaining: 0us
0:	learn: 0.0673693	total: 802ms	remaining: 54.5s
15:	learn: 0.0673641	total: 12.7s	remaining:

In [3]:
test = pd.read_parquet('/kaggle/input/avitotech-ml-cup-2024-test-dataset/test.parquet')
test.head(10)

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main
0,2714742,3026,2,4056,2024-09-23,8,True
1,2714742,2994,2,3954,2024-09-23,8,True
2,2714742,97,2,1752,2024-09-23,8,True
3,2714742,3539,2,1244,2024-09-23,8,True
4,2714742,2756,2,2003,2024-09-23,8,True
5,2714742,3893,2,3359,2024-09-23,8,True
6,410633,4136,2,2198,2024-09-23,8,True
7,410633,1072,2,1667,2024-09-23,8,True
8,410633,3744,2,447,2024-09-23,8,True
9,410633,1611,2,3690,2024-09-23,8,True


In [4]:
campaigns = pd.read_parquet('/kaggle/input/avitotech-ml-cup-2024-campaigns-meta-dataset/campaigns_meta.parquet')
campaigns.head(10)

,adv_campaign_id,start_date,end_date,goal_cost,goal_budget,location_id,logcat_id
0,2153,2024-09-21,2024-10-02,6.661659,9429.056096,70,59
1,3103,2024-09-10,2024-09-16,2.853378,3844.482933,30,40
2,2816,2024-09-10,2024-09-17,3.058230,1455.156612,56,65
3,3603,2024-09-10,2024-09-16,4.395015,2592.232475,30,50
4,1328,2024-09-10,2024-09-16,3.891329,2836.139672,30,51
5,2004,2024-09-10,2024-09-16,3.228728,278.431978,41,51
6,1906,2024-09-11,2024-09-17,18.486252,3544.500596,60,37
7,89,2024-09-11,2024-09-17,8.576056,20636.439193,20,18
8,3312,2024-09-11,2024-09-17,3.709471,3569.102100,60,65
9,828,2024-09-11,2024-09-17,5.643515,4882.446426,30,51


In [5]:
# doing the same as with train dataset
test['event_date'] = test['event_date'].astype('datetime64[ns]')
campaigns['start_date'] = campaigns['start_date'].astype('datetime64[ns]')
campaigns['end_date'] = campaigns['end_date'].astype('datetime64[ns]')
test = test.merge(campaigns, on='adv_campaign_id', how='inner')
del campaigns
test['start_date'] = ((test['event_date'] - test['start_date']) / 86400000000000).astype(int)
test['end_date'] = ((test['end_date'] - test['event_date']) / 86400000000000).astype(int)
test['goal_cost'] = test['goal_budget'] / test['goal_cost']
test = test.drop(columns=['event_date']) \
           .rename(columns={
               'start_date': 'days_from_start',
               'end_date'  : 'days_to_end',
               'goal_cost' : 'goals_number'
           })
test.head(10)

,user_id,adv_campaign_id,platform_id,adv_creative_id,banner_code,is_main,days_from_start,days_to_end,goals_number,goal_budget,location_id,logcat_id
0,2714742,3026,2,4056,8,True,3,3,1068.546422,5080.923372,1,65
1,2714742,2994,2,3954,8,True,4,10,7131.705019,55768.516996,30,65
2,2714742,97,2,1752,8,True,3,3,1950.890659,10024.341068,1,65
3,2714742,3539,2,1244,8,True,3,3,1962.145426,9925.292320,1,65
4,2714742,2756,2,2003,8,True,5,4,3996.304353,19293.322118,30,65
5,2714742,3893,2,3359,8,True,3,26,2193.393951,8696.657594,27,65
6,410633,4136,2,2198,8,True,4,2,1621.155625,4019.976115,1,40
7,410633,1072,2,1667,8,True,6,0,4750.251884,13515.052035,46,40
8,410633,3744,2,447,8,True,3,3,1524.018222,3736.664709,46,40
9,410633,1611,2,3690,8,True,4,10,9095.618642,28401.774432,1,40


In [6]:
sample = pd.read_csv('/kaggle/input/avitotech-ml-cup-2024-sample-submission/sample_submission.csv')
sample.head(10)

,user_id,adv_campaign_id,predict
0,2714742,3026,0.5
1,2714742,2994,0.5
2,2714742,97,0.5
3,2714742,3539,0.5
4,2714742,2756,0.5
5,2714742,3893,0.5
6,410633,4136,0.5
7,410633,1072,0.5
8,410633,3744,0.5
9,410633,1611,0.5


In [7]:
predict1 = model.predict(test)
sample1 = sample.copy()
sample1['predict'] = predict1
sample1.to_csv('sample1.csv', index=False)
print('sample1 has gone successfully')

sample1 has gone successfully


In [8]:
_min = min(predict1)
_max = max(predict1)
predict2 = list(map(lambda x: (x - _min) / (_max - _min), predict1))
sample2 = sample.copy()
sample2['predict'] = predict2
sample2.to_csv('sample2.csv', index=False)
print('sample2 has gone successfully')

sample2 has gone successfully
